# Annotate features for proteins.

## Step1 Annotate pLI and pLoF score for valid proteins. 

### Step1.1 Extract all protein molecules from a filtered LPPI

In [1]:
import pandas as pd

inter_file = 'lppi_with_valid_lnc.csv'
inter = pd.read_csv(inter_file)

# Concatenate two columns into a new Series and remove duplicates
molecule = pd.concat([inter['Node_i'], inter['Node_j']]).reset_index(drop=True)
molecule_df = pd.DataFrame(molecule, columns=['molecule'])
molecule_df = molecule_df.drop_duplicates()

protein_file = '../../data/LPPI/human/protein.csv'
proteins = pd.read_csv(protein_file)

proteins = proteins[proteins['protein_id'].isin(molecule_df['molecule'])]

# Export to CSV file
protein_file = 'proteins.csv'
proteins.to_csv(protein_file, index=False)


### Step1.2 Annotate pLI score for valid proteins.

In [6]:
import pandas as pd

valid_protein = pd.read_csv("proteins.csv", dtype=str)
pLoF = pd.read_csv("../../omics/protein/human/pLoF_v2.txt", sep='\t')

# Filtering required columns from pLoF
pLoF = pLoF[['gene', 'obs_lof', 'exp_lof', 'oe_lof', 'lof_z', 'pLI']]
#pLoF = pLoF[['gene', 'lof.obs', 'lof.exp', 'lof.oe', 'lof.z_score', 'lof.pLI']]

# Merging on 'protein' column from valid_protein and 'gene' column from pLoF
protein_pLI = pd.merge(valid_protein, pLoF, left_on='protein', right_on='gene', how='inner')
protein_pLI = protein_pLI.drop(columns=['gene', 'protein'])

# Extract rows containing NaN values in protein_pLI
na_score_protein = protein_pLI[protein_pLI.isna().any(axis=1)]
na_score_protein = na_score_protein[['protein_id']]

protein_pLI_cleaned = protein_pLI.dropna()
protein_invalid_score = valid_protein[~valid_protein['protein_id'].isin(protein_pLI_cleaned['protein_id'])]

pLI_means = protein_pLI_cleaned.groupby('protein_id').mean().reset_index()

# Save results
pLI_means.to_csv('protein_annotation.csv', index=False)
na_score_protein.to_csv('NA_score_protein.csv', index=False)  # Saving rows with NaN values
protein_invalid_score.to_csv('invalid_score_protein.csv', index=False)  # Saving rows with NaN values


### Step1.3 Convert oe_lof&lof_z to p-value and get log.

In [7]:
import pandas as pd
import numpy as np
from scipy.stats import poisson, norm

# Load the annotation
df = pd.read_csv("protein_annotation.csv")

# Calculate p-value for oe_lof using the Poisson distribution
df["oe_lof_pval"] = df.apply(lambda row: poisson.cdf(row['obs_lof'], row['exp_lof']), axis=1)

# Calculate p-value for lof_z using a one-tailed Z-test
df["lof_z_pval"] = df["lof_z"].apply(lambda x: 1 - norm.cdf(x) if pd.notna(x) else np.nan)

# Apply a log10 transformation to p-values with a small number adjustment
eps = 1e-300
df["log_oe_lof_pval"] = -np.log10(df["oe_lof_pval"].clip(lower=eps))
df["log_lof_z_pval"]  = -np.log10(df["lof_z_pval"].clip(lower=eps))
df["log_pLI"] = -np.log10((1 - df["pLI"]).clip(lower=eps))

# Select columns to keep
df = df[['protein_id', "log_oe_lof_pval", "log_lof_z_pval", "log_pLI"]]

# Save the transformed annotation
df.to_csv("transformed_protein_annotation.csv", index=False)


### Step1.4 Delete interaction with protein which have NA pLI score.

In [10]:
import pandas as pd

inter = pd.read_csv("lppi_with_valid_lnc.csv")
invalid_protein = pd.read_csv("invalid_score_protein.csv")
NA_protein = pd.read_csv("NA_score_protein.csv")

inter = inter[~inter['Node_i'].isin(invalid_protein['protein_id'])]
inter = inter[~inter['Node_j'].isin(invalid_protein['protein_id'])]

inter = inter[~inter['Node_i'].isin(NA_protein['protein_id'])]
inter = inter[~inter['Node_j'].isin(NA_protein['protein_id'])]

tissues = ['heart','lung','stomach']
for tissue in tissues:
	lnc = pd.read_csv(f"{tissue}_annotation.csv")
	lnc = lnc[lnc['lncRNA_id'].isin(inter['Node_i'])]
	lnc.to_csv(f"valid_{tissue}_annotation.csv", index=False)

inter.columns = ['source', 'target', 'weight']
inter.to_csv("valid_inter.csv", index=False)



/tmp/ipykernel_1487119/3340504017.py:15: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  lnc = pd.read_csv(f"{tissue}_annotation.csv")
/tmp/ipykernel_1487119/3340504017.py:15: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  lnc = pd.read_csv(f"{tissue}_annotation.csv")
/tmp/ipykernel_1487119/3340504017.py:15: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  lnc = pd.read_csv(f"{tissue}_annotation.csv")
